In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import time

In [18]:
class DataPreparation:
    def __init__(self, data_path):
        self.data_path = data_path
        self.feature_tensor = None
        self.target_tensor = None
        self.df = None
        self.dataloader = None
    
    def load_data(self):
        self.df = pd.read_csv(self.data_path)
    
    def data_preparation(self):
        selected_features = ['dst_host_same_src_port_rate',
                             'is_guest_login',
                             'num_failed_logins',
                             'dst_host_diff_srv_rate',
                             'rerror_rate',
                             'logged_in',
                             'serror_rate',
                             'same_srv_rate',
                             'protocol_type_0',
                             'protocol_type_1',
                             'dst_host_srv_diff_host_rate']

        label_encoder = LabelEncoder()
        self.df['label'] = label_encoder.fit_transform(self.df['label'])
        self.df = self.df.astype({col: 'float32' for col in self.df.select_dtypes(include=['int', 'float64']).columns})

        self.feature_tensor = torch.tensor(self.df[selected_features].values)
        self.target_tensor = torch.tensor(self.df['label'].values, dtype=torch.long)

        dataset = TensorDataset(self.feature_tensor, self.target_tensor)
        self.dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [19]:
class MultiLayerPerceptron(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 300),
            nn.ReLU(),
            nn.Linear(300, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 38),
        )
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [20]:
class ModelTrainer:
    def __init__(self, model, dataloader):
        device = torch.device("mps")
        self.model = model.to(device)
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=1e-3)
        self.dataloader = dataloader
        self.device = torch.device("mps")

    def train(self, epochs):
        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            for batch, (X, y) in enumerate(self.dataloader):
                X, y = X.to(self.device), y.to(self.device)
                prediction = self.model(X)
                loss = self.loss_fn(prediction, y)
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()

    def test(self):
        self.model.eval()
        size = len(self.dataloader.dataset)
        num_batches = len(self.dataloader)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in self.dataloader:
                X, y = X.to(self.device), y.to(self.device)
                pred = self.model(X)
                test_loss += self.loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [21]:
data_preparation = DataPreparation("../../../transformed_datasets/transformed_KDD_CUPP99_2.csv")
data_preparation.load_data()
data_preparation.data_preparation()

In [22]:
model = MultiLayerPerceptron()
trainer = ModelTrainer(model, data_preparation.dataloader)

In [24]:
t0 = time.time()
trainer.train(epochs=100)
training_time = time.time() - t0
print(training_time)
trainer.test()

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78